In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup 
import pandas as pd
import numpy as np
import re
import requests
from random import *
from tqdm import tqdm #progress bar
from datetime import datetime
import os
import glob
import pickle #for saving data

In [2]:
make_model_ads_data = pd.read_pickle("data/make_model_ads_links_concatinated.pkl")

In [3]:
make_model_ads_data

,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_160748,Skoda,22900,Roomster,13,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
133,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
134,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
135,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
136,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...


In [4]:
latest_scrape = make_model_ads_data.groupby(['car_make', 'car_model'], dropna=False).agg(number_of_ads=('ad_link', 'count'), latest_scrape=('download_date_time', 'max'))
latest_scrape = latest_scrape.reset_index()
latest_scrape

,car_make,car_model,number_of_ads,latest_scrape
0,Aston Martin,Rapide,49,20220117_174327
1,BMW,535,901,20220117_173238
2,BMW,732,5,20220117_175728
3,BMW,735,138,20220118_144643
4,BMW,214 Gran Tourer,2,20220117_162428
5,BMW,M4,880,20220118_144517
6,BMW,X6,943,20220117_163645
7,Bentley,Continental GTC,167,20220117_175658
8,Chevrolet,Matiz,317,20220117_162756
9,Ferrari,GTC4Lusso,73,20220117_182631


In [5]:
make_model_ads_data_latest = pd.merge(make_model_ads_data, latest_scrape[['car_make', 'car_model', 'latest_scrape']], how = 'left', left_on = ['car_make', 'car_model'], right_on = ['car_make', 'car_model'])
make_model_ads_data_latest = make_model_ads_data_latest.reset_index(drop=True)
# keep rows where download_date_time is equal to latest_scrape
make_model_ads_data_latest = make_model_ads_data_latest[make_model_ads_data_latest['download_date_time'] == make_model_ads_data_latest['latest_scrape']]
make_model_ads_data_latest = make_model_ads_data_latest.reset_index(drop=True)
# drop the latest_scrape column
make_model_ads_data_latest = make_model_ads_data_latest.drop(columns = ['latest_scrape'])
make_model_ads_data_latest

,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162428,BMW,3500,214 Gran Tourer,116,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162428,BMW,3500,214 Gran Tourer,116,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162448,Mercedes-Benz,17200,CLS 400 Shooting Brake,263,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162448,Mercedes-Benz,17200,CLS 400 Shooting Brake,263,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162448,Mercedes-Benz,17200,CLS 400 Shooting Brake,263,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
8562,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
8563,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
8564,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
8565,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...


In [6]:
def get_ad_data(ad_link = '', sleep_time = 5, save_to_csv = True, save_to_pickle = True):
    
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.managed_default_content_settings.images": 2} # this is to not load images
    chrome_options.add_experimental_option("prefs", prefs)

    #start a driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)

    #get the number of pages
    driver.get(ad_link)
    time.sleep(sleep_time)
    ad_source = driver.page_source
    ad_soup = BeautifulSoup(ad_source, 'html.parser')
    driver.close()

    try:
        table_pre = ad_soup.find("div", { "class" : "cBox-body cBox-body--technical-data"})
        all_div = table_pre.findAll("div", { "class" : re.compile('^g-col-6')})
    except:
        table_pre = []
        all_div = []
    
    description_list = []
    value_list = []

    try:
        div_length = len(all_div)
    except:
        div_length = 2

    i = 1
    for i in range(div_length - 1):
        try:
            description_list.append(all_div[i].text)
            value_list.append(all_div[i+1].text)
            i += 2
        except:
            description_list.append('no_description')
            value_list.append('no_value')

    #create a dataframe
    df = pd.DataFrame(list(zip(description_list, value_list)), columns = ['description', 'value'])

    #keep rows where value is equal to the followings
    data_we_want_to_keep = ['Preis', 'Kategorie', 'Kilometerstand', 'Hubraum', 'Leistung', 'Kraftstoffart', 'Anzahl Sitzplätze', 'Getriebe', 'Erstzulassung', 'Farbe', 'Innenausstattung']
    df = df[df['description'].isin(data_we_want_to_keep)]

    # #transpose with description as column names
    #df = df.T
    df = df.set_index('description').T.reset_index(drop=True)
    df = df.rename_axis(None, axis=1)
    df['link'] = ad_link

    #datetime string
    now = datetime.now() 
    datetime_string = str(now.strftime("%Y%m%d_%H%M%S"))

    df['download_date_time'] = datetime_string

    #save the dataframe if save_to_csv is True
    if save_to_csv:
        #check if folder exists and if not create it
        if not os.path.exists('data/make_model_ads_data'):
            os.makedirs('data/make_model_ads_data')

        df.to_csv(str('data/make_model_ads_data/links_on_one_page_df' + datetime_string + '.csv'), index = False)

    if save_to_pickle:
        if not os.path.exists('data/make_model_ads_data'):
            os.makedirs('data/make_model_ads_data')
        
        df.to_pickle('data/make_model_ads_data/links_on_one_page_df' + datetime_string + "pkl")

    return(df)

In [7]:
# len_of_links = len(make_model_ads_data_latest)

# for i in tqdm(range(20)):
#     ad_link = make_model_ads_data_latest['ad_link'][i]
#     data = get_ad_data(ad_link = ad_link, sleep_time = 5, save_to_csv = True, save_to_pickle = True)

In [8]:
# concatenate the dataframes in one folder to get one file (with different columns)
def concatenate_dfs(indir, save_to_csv = True, save_to_pickle = True):
    

    fileList=glob.glob(str(str(indir) + "*.csv"))
    print("Found this many CSVs: ", len(fileList), " In this folder: ", str(os.getcwd()) + "/" + str(indir))

    output_file = pd.concat([pd.read_csv(filename) for filename in fileList])

    if save_to_csv:
        output_file.to_csv("data/make_model_ads_concatinated.csv", index=False)

    if save_to_pickle:
        output_file.to_pickle("data/make_model_ads_concatinated.pkl")

    output_file.reset_index(drop=True, inplace=True)

    return(output_file)

In [9]:
individual_ads_data = concatenate_dfs(indir= "data/make_model_ads_data/", save_to_csv = False, save_to_pickle = True)

Found this many CSVs:  70  In this folder:  c:\Users\menyh\Documents\mobile_de_scraping/data/make_model_ads_data/


In [10]:
individual_ads_data

,Preis,Kategorie,Kilometerstand,Hubraum,Leistung,Kraftstoffart,Anzahl Sitzplätze,Getriebe,Erstzulassung,Farbe,Innenausstattung,link,download_date_time
0,14.990 €Ohne BewertungKeine realistische Preis...,Van / Minibus,73.800 km,1.496 cm³,70 kW (95 PS),Diesel,5.0,Schaltgetriebe,10/2015,Grau Metallic,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173844
1,15.500 €Ohne BewertungKeine realistische Preis...,Van / Minibus,29.000 km,1.499 cm³,100 kW (136 PS),Benzin,5.0,Schaltgetriebe,06/2015,NaN,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173914
2,41.990 €Ohne BewertungKeine realistische Preis...,Kombi,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173924
3,28.970 €Ohne BewertungKeine realistische Preis...,Kombi,152.125 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2014,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173934
4,33.950 €Ohne BewertungKeine realistische Preis...,Kombi,107.903 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,04/2017,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,1.990 €Fairer PreisZur Preisbewertung,Kleinwagen,97.000 km,995 cm³,49 kW (67 PS),"Benzin, E10-geeignet",5.0,Schaltgetriebe,07/2008,Silber Metallic,Grau,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_183038
66,2.790 €Ohne BewertungKeine realistische Preisb...,Kleinwagen,124.000 km,995 cm³,49 kW (67 PS),Benzin,5.0,Schaltgetriebe,09/2009,Blau Metallic,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_183052
67,2.799 €Fairer PreisZur Preisbewertung,Kleinwagen,96.600 km,995 cm³,49 kW (67 PS),Benzin,5.0,Schaltgetriebe,01/2010,Schwarz,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_183104
68,1.999 €Guter PreisZur Preisbewertung,Kleinwagen,88.000 km,796 cm³,38 kW (52 PS),Benzin,5.0,Schaltgetriebe,03/2008,Blau,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_183132


We can do one more thing: We can check whether we already scraped a specific ad and scrape only that we have not. This way, if there is an error we can continue our scraping where we left.

In [11]:
len_of_links = len(make_model_ads_data_latest)
# get random numbers between 0 and len_of_links
random_numbers = np.random.randint(0, len_of_links, size = 20)

j = 0
for i in tqdm(random_numbers):
    ad_link = make_model_ads_data_latest['ad_link'][i]

    if ad_link not in make_model_ads_data['link'].values:
        data = get_ad_data(ad_link = ad_link, sleep_time = 5, save_to_csv = True, save_to_pickle = True)
        j = j + 1
        
print("scraped ", j, " new ads")

  0%|          | 0/20 [00:00<?, ?it/s]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
<ipython-input-6-eacc37791912>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
  5%|▌         | 1/20 [00:15<04:45, 15.02s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 10%|█         | 2/20 [00:25<03:37, 12.06s/it]

====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
 15%|█▌        | 3/20 [00:36<03:

scraped  20  new ads


We can do some analysis on the top of the data.

First merge the make and model to the adds

In [12]:
individual_ads_data = concatenate_dfs(indir= "data/make_model_ads_data/", save_to_csv = False, save_to_pickle = False)

Found this many CSVs:  90  In this folder:  c:\Users\menyh\Documents\mobile_de_scraping/data/make_model_ads_data/


In [13]:
individual_ads_data

,Preis,Kategorie,Kilometerstand,Hubraum,Leistung,Kraftstoffart,Anzahl Sitzplätze,Getriebe,Erstzulassung,Farbe,Innenausstattung,link,download_date_time
0,14.990 €Ohne BewertungKeine realistische Preis...,Van / Minibus,73.800 km,1.496 cm³,70 kW (95 PS),Diesel,5.0,Schaltgetriebe,10/2015,Grau Metallic,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173844
1,15.500 €Ohne BewertungKeine realistische Preis...,Van / Minibus,29.000 km,1.499 cm³,100 kW (136 PS),Benzin,5.0,Schaltgetriebe,06/2015,NaN,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173914
2,41.990 €Ohne BewertungKeine realistische Preis...,Kombi,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173924
3,28.970 €Ohne BewertungKeine realistische Preis...,Kombi,152.125 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2014,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173934
4,33.950 €Ohne BewertungKeine realistische Preis...,Kombi,107.903 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,04/2017,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_173944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_185818
86,5.250 €Fairer PreisZur Preisbewertung,Van / Minibus,136.660 km,1.197 cm³,63 kW (86 PS),"Benzin, E10-geeignet",5.0,Schaltgetriebe,08/2011,Rot Metallic,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_185828
87,2.150 €Guter PreisZur Preisbewertung,Van / Minibus,209.000 km,1.422 cm³,51 kW (69 PS),Diesel,5.0,Schaltgetriebe,03/2008,Schwarz,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_185839
88,20.750 €Guter PreisZur Preisbewertung,Van / Minibus,116.879 km,1.997 cm³,110 kW (150 PS),Diesel,7.0,Schaltgetriebe,10/2017,Grau Metallic,"Stoff, Grau",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_185850


In [14]:
make_model_ads_data_latest

,ad_link,make_model_link,download_date_time,car_make,id1,car_model,id2,link
0,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162428,BMW,3500,214 Gran Tourer,116,https://suchen.mobile.de/fahrzeuge/search.html...
1,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162428,BMW,3500,214 Gran Tourer,116,https://suchen.mobile.de/fahrzeuge/search.html...
2,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162448,Mercedes-Benz,17200,CLS 400 Shooting Brake,263,https://suchen.mobile.de/fahrzeuge/search.html...
3,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162448,Mercedes-Benz,17200,CLS 400 Shooting Brake,263,https://suchen.mobile.de/fahrzeuge/search.html...
4,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220117_162448,Mercedes-Benz,17200,CLS 400 Shooting Brake,263,https://suchen.mobile.de/fahrzeuge/search.html...
...,...,...,...,...,...,...,...,...
8562,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
8563,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
8564,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...
8565,https://suchen.mobile.de/fahrzeuge/details.htm...,https://suchen.mobile.de/fahrzeuge/search.html...,20220118_144643,BMW,3500,735,37,https://suchen.mobile.de/fahrzeuge/search.html...


In [15]:
# merge the individual ads data with the make_model_ads_data_latest and keep only the latest download date
def merge_make_model_keep_latest(data = individual_ads_data):
    latest_scrape = data.groupby(['link'], dropna=True).agg(number_of_ads=('link', 'count'), latest_scrape=('download_date_time', 'max'))
    latest_scrape = latest_scrape.reset_index()

    data = pd.merge(data, latest_scrape[['link', 'latest_scrape']], how = 'left', left_on = ['link'], right_on = ['link'])
    data = data.reset_index(drop=True)
    # keep rows where download_date_time is equal to latest_scrape
    data = data[data['download_date_time'] == data['latest_scrape']]
    data = data.reset_index(drop=True)
    # drop the latest_scrape column
    data = data.drop(columns = ['latest_scrape'])
    
    data = pd.merge(data, make_model_ads_data_latest[['ad_link', 'car_make', 'car_model']], how = 'left', left_on = 'link', right_on = 'ad_link')

    return data

In [16]:
ads_df = merge_make_model_keep_latest(data = individual_ads_data)

In [17]:
ads_df

,Preis,Kategorie,Kilometerstand,Hubraum,Leistung,Kraftstoffart,Anzahl Sitzplätze,Getriebe,Erstzulassung,Farbe,Innenausstattung,link,download_date_time,ad_link,car_make,car_model
0,14.990 €Ohne BewertungKeine realistische Preis...,Van / Minibus,73.800 km,1.496 cm³,70 kW (95 PS),Diesel,5.0,Schaltgetriebe,10/2015,Grau Metallic,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182248,https://suchen.mobile.de/fahrzeuge/details.htm...,BMW,214 Gran Tourer
1,15.500 €Ohne BewertungKeine realistische Preis...,Van / Minibus,29.000 km,1.499 cm³,100 kW (136 PS),Benzin,5.0,Schaltgetriebe,06/2015,NaN,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182257,https://suchen.mobile.de/fahrzeuge/details.htm...,BMW,214 Gran Tourer
2,41.990 €Ohne BewertungKeine realistische Preis...,Kombi,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182307,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
3,28.970 €Ohne BewertungKeine realistische Preis...,Kombi,152.125 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2014,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182323,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
4,33.950 €Ohne BewertungKeine realistische Preis...,Kombi,107.903 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,04/2017,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182346,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
5,38.880 €Ohne BewertungKeine realistische Preis...,Kombi,90.000 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,06/2015,Braun Metallic,"Vollleder, Braun",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182356,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
6,38.880 €Ohne BewertungKeine realistische Preis...,Kombi,90.000 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,06/2015,Braun Metallic,"Vollleder, Braun",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182406,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
7,41.990 €Ohne BewertungKeine realistische Preis...,Sportwagen / Coupé,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182416,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
8,41.990 €Ohne BewertungKeine realistische Preis...,Kombi,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182425,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake
9,42.890 €Ohne BewertungKeine realistische Preis...,Kombi,106.003 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,04/2018,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182437,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake


In [18]:
def clean_data(data):
    data['price'] = data['Preis'].str.replace('.', '')
    data['price'] = data['price'].str.extract('(\d+)').astype(int, errors='ignore')

    data = data[~data['price'].isna()]
    data['price'] = data['price'].astype(int)

    return data

In [19]:
ads_df_clean = clean_data(data = ads_df)
ads_df_clean

<ipython-input-18-dd86c7f8e7fe>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data['price'] = data['Preis'].str.replace('.', '')
<ipython-input-18-dd86c7f8e7fe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['price'].astype(int)


,Preis,Kategorie,Kilometerstand,Hubraum,Leistung,Kraftstoffart,Anzahl Sitzplätze,Getriebe,Erstzulassung,Farbe,Innenausstattung,link,download_date_time,ad_link,car_make,car_model,price
0,14.990 €Ohne BewertungKeine realistische Preis...,Van / Minibus,73.800 km,1.496 cm³,70 kW (95 PS),Diesel,5.0,Schaltgetriebe,10/2015,Grau Metallic,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182248,https://suchen.mobile.de/fahrzeuge/details.htm...,BMW,214 Gran Tourer,14990
1,15.500 €Ohne BewertungKeine realistische Preis...,Van / Minibus,29.000 km,1.499 cm³,100 kW (136 PS),Benzin,5.0,Schaltgetriebe,06/2015,NaN,Stoff,https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182257,https://suchen.mobile.de/fahrzeuge/details.htm...,BMW,214 Gran Tourer,15500
2,41.990 €Ohne BewertungKeine realistische Preis...,Kombi,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182307,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,41990
3,28.970 €Ohne BewertungKeine realistische Preis...,Kombi,152.125 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2014,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182323,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,28970
4,33.950 €Ohne BewertungKeine realistische Preis...,Kombi,107.903 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,04/2017,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182346,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,33950
5,38.880 €Ohne BewertungKeine realistische Preis...,Kombi,90.000 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,06/2015,Braun Metallic,"Vollleder, Braun",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182356,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,38880
6,38.880 €Ohne BewertungKeine realistische Preis...,Kombi,90.000 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,06/2015,Braun Metallic,"Vollleder, Braun",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182406,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,38880
7,41.990 €Ohne BewertungKeine realistische Preis...,Sportwagen / Coupé,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182416,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,41990
8,41.990 €Ohne BewertungKeine realistische Preis...,Kombi,62.100 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,12/2016,Grau Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182425,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,41990
9,42.890 €Ohne BewertungKeine realistische Preis...,Kombi,106.003 km,3.498 cm³,245 kW (333 PS),Benzin,5.0,Automatik,04/2018,Schwarz Metallic,"Vollleder, Schwarz",https://suchen.mobile.de/fahrzeuge/details.htm...,20220119_182437,https://suchen.mobile.de/fahrzeuge/details.htm...,Mercedes-Benz,CLS 400 Shooting Brake,42890


In [20]:
mean_price_of_models = ads_df_clean.groupby(['car_make', 'car_model'], dropna=False).agg(number_of_ads=('price', 'count'), mean_price=('price', 'mean'))
mean_price_of_models = mean_price_of_models.reset_index()
#round mean_price
mean_price_of_models['mean_price'] = mean_price_of_models['mean_price'].round(0).astype(int)
mean_price_of_models

,car_make,car_model,number_of_ads,mean_price
0,BMW,535,2,20074
1,BMW,735,1,12000
2,BMW,214 Gran Tourer,2,15245
3,BMW,M4,1,73400
4,BMW,X6,2,36485
5,Chevrolet,Matiz,16,2810
6,Ford,Galaxy,3,24377
7,Ford,Grand Tourneo,1,22990
8,Mercedes-Benz,CLS 400 Shooting Brake,20,37322
9,Mercedes-Benz,S 350,3,37663
